In [10]:
# Import the essential libraries for data manipulation and financial data sourcing
import pandas as pd
import numpy as np
import yfinance as yf
import os
import pandas_datareader.data as web # For Fama-French Data
import datetime

print("Libraries imported successfully.")


Libraries imported successfully.


In [2]:
# --- Define our investment universe and the time period for our analysis ---
tickers = ['AAPL', 'MSFT', 'JPM', 'JNJ', 'XOM', 'PG', 'GOOGL', 'AMZN', 'UNH', 'TSLA']
start_date = "2019-01-01"
end_date = "2023-12-31"

# --- Define file paths for our data pipeline ---
DATA_DIR = 'data'
RAW_PRICES_FILE = os.path.join(DATA_DIR, 'raw_adj_close_prices.csv')
MONTHLY_PRICES_FILE = os.path.join(DATA_DIR, 'monthly_prices.csv')
MONTHLY_EXCESS_RETURNS_FILE = os.path.join(DATA_DIR, 'monthly_excess_returns.csv')

# --- Create the data directory if it doesn't exist ---
if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)
    print(f"Created directory: {DATA_DIR}")

print("Universe and file paths defined.")


Universe and file paths defined.


In [33]:
# --- Download Fama-French Risk-Free Rate Data ---
# I get the monthly risk-free rate (RF) from the F-F_Research_Data_Factors dataset.
print("Downloading Fama-French risk-free rate data...")
start = datetime.datetime(2019,1,1)
end = datetime.datetime(2023,12,31)
ff_data = web.DataReader('F-F_Research_Data_Factors', 'famafrench', start=start, end=end)[0]
rf_monthly = (ff_data['RF'] / 100).to_frame(name = 'rf_rate')

# convert the date index to timestamp
rf_monthly.index = rf_monthly.index.to_timestamp('M')

print("Fama-French data downloaded successfully.")
rf_monthly.head()

Fama-French data downloaded successfully.


C:\Users\HoNad\AppData\Local\Temp\ipykernel_21132\3721148468.py:6: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff_data = web.DataReader('F-F_Research_Data_Factors', 'famafrench', start=start, end=end)[0]
C:\Users\HoNad\AppData\Local\Temp\ipykernel_21132\3721148468.py:6: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff_data = web.DataReader('F-F_Research_Data_Factors', 'famafrench', start=start, end=end)[0]


,rf_rate
Date,
2019-01-31,0.0021
2019-02-28,0.0018
2019-03-31,0.0019
2019-04-30,0.0021
2019-05-31,0.0021


In [34]:
rf_monthly.head()

,rf_rate
Date,
2019-01-31,0.0021
2019-02-28,0.0018
2019-03-31,0.0019
2019-04-30,0.0021
2019-05-31,0.0021


In [35]:
# --- Acquire Stock Price Data ---
if not os.path.exists(RAW_PRICES_FILE):
    print("Raw price data not found. Downloading from yfinance...")
    
    # Download the dividend and split-adjusted prices
    adj_close_prices = yf.download(tickers, 
                                   start=start_date, 
                                   end=end_date, 
                                   auto_adjust=True)['Close']
    
    # Save the raw downloaded data for future runs
    adj_close_prices.to_csv(RAW_PRICES_FILE)
    print(f"Data downloaded and saved to {RAW_PRICES_FILE}")
else:
    print(f"Loading raw price data from local file: {RAW_PRICES_FILE}...")
    adj_close_prices = pd.read_csv(RAW_PRICES_FILE, index_col='Date', parse_dates=True)
    print("Data loaded successfully.")

print("\nRaw Adjusted Close Prices:")
adj_close_prices.head()


Loading raw price data from local file: data\raw_adj_close_prices.csv...
Data loaded successfully.

Raw Adjusted Close Prices:


,AAPL,AMZN,GOOGL,JNJ,JPM,MSFT,PG,TSLA,UNH,XOM
Date,,,,,,,,,,
2019-01-02,37.538811,76.956497,52.372784,105.609138,81.616707,94.789688,76.454651,20.674667,218.587982,50.775375
2019-01-03,33.799667,75.014000,50.922283,103.930931,80.456787,91.302574,75.918587,20.024000,212.627121,49.995796
2019-01-04,35.242558,78.769501,53.534264,105.675262,83.422859,95.548973,77.468117,21.179333,215.113846,51.839119
2019-01-07,35.164112,81.475502,53.427505,104.997391,83.480850,95.670830,77.158226,22.330667,215.526703,52.108704
2019-01-08,35.834457,82.829002,53.896770,107.436096,83.323418,96.364494,77.442993,22.356667,218.408463,52.487564


In [28]:
# --- Process Prices and Calculate Returns ---

# Ensure there are no missing values
adj_close_prices.ffill(inplace=True)
adj_close_prices.bfill(inplace=True)

# Resample daily prices to get month-end prices
monthly_prices = adj_close_prices.resample('ME').last()

# Calculate monthly total returns
monthly_returns = monthly_prices.pct_change()

# Drop the first row which will be NaN
monthly_returns.dropna(axis=0, how='all', inplace=True)

print("Monthly total returns calculated.")
monthly_returns.head()


Monthly total returns calculated.


,AAPL,AMZN,GOOGL,JNJ,JPM,MSFT,PG,TSLA,UNH,XOM
Date,,,,,,,,,,
2019-02-28,0.044777,-0.045906,0.000586,0.033560,0.008309,0.077358,0.021561,0.041886,-0.103553,0.090440
2019-03-31,0.097026,0.085936,0.044685,0.023053,-0.029992,0.052754,0.055809,-0.125109,0.024704,0.022396
2019-04-30,0.056436,0.081859,0.018753,0.010087,0.155171,0.107343,0.030578,-0.147109,-0.057389,-0.006436
2019-05-31,-0.124213,-0.078613,-0.077117,-0.064820,-0.086945,-0.049481,-0.033527,-0.224266,0.037456,-0.108356
2019-06-30,0.130519,0.066792,-0.021419,0.061990,0.055116,0.083118,0.065494,0.206848,0.013609,0.082804


In [38]:
# --- Calculate Monthly Excess Returns ---

# We use an inner join to ensure we only have dates where both datasets are available
# This aligns the stock returns and risk-free rates perfectly by date.
merged_data = monthly_returns.join(rf_monthly, how='inner')

# Calculate excess return by subtracting the risk-free rate from each stock's return
monthly_excess_returns = merged_data[tickers].subtract(merged_data['rf_rate'], axis=0)
print("Monthly excess returns calculated.")
monthly_excess_returns.head()

Monthly excess returns calculated.


,AAPL,MSFT,JPM,JNJ,XOM,PG,GOOGL,AMZN,UNH,TSLA
Date,,,,,,,,,,
2019-02-28,0.042977,0.075558,0.006509,0.031760,0.088640,0.019761,-0.001214,-0.047706,-0.105353,0.040086
2019-03-31,0.095126,0.050854,-0.031892,0.021153,0.020496,0.053909,0.042785,0.084036,0.022804,-0.127009
2019-04-30,0.054336,0.105243,0.153071,0.007987,-0.008536,0.028478,0.016653,0.079759,-0.059489,-0.149209
2019-05-31,-0.126313,-0.051581,-0.089045,-0.066920,-0.110456,-0.035627,-0.079217,-0.080713,0.035356,-0.226366
2019-06-30,0.128719,0.081318,0.053316,0.060190,0.081004,0.063694,-0.023219,0.064992,0.011809,0.205048


In [39]:
# --- Save Processed Data for Next Notebooks ---

monthly_prices.to_csv(MONTHLY_PRICES_FILE)
monthly_excess_returns.to_csv(MONTHLY_EXCESS_RETURNS_FILE)

print(f"Final output files saved to '{DATA_DIR}' directory.")
print("Notebook 1 is complete.")


Final output files saved to 'data' directory.
Notebook 1 is complete.
